# Test d'extraction d'entités et relations

Ce notebook teste l'extraction d'entités et de relations depuis les documents.

In [ ]:
import sys
sys.path.append('..')

import json
from pathlib import Path
from src.extraction.entity_extractor import EntityExtractor
from src.extraction.relation_extractor import RelationExtractor
import matplotlib.pyplot as plt
import pandas as pd

## 1. Charger les documents traités

In [ ]:
# Charger les documents
with open("../data/processed/documents.json", 'r') as f:
    documents = json.load(f)

print(f"Nombre de documents chargés: {len(documents)}")

# Prendre un échantillon pour les tests
sample_docs = documents[:5]  # Premier 5 documents
print(f"Échantillon de {len(sample_docs)} documents pour les tests")

## 2. Test d'extraction d'entités

In [ ]:
# Initialiser l'extracteur
entity_extractor = EntityExtractor()

# Extraire les entités d'un document
test_doc = sample_docs[0]
print(f"Document test: {test_doc['filename']}")
print(f"Longueur du texte: {len(test_doc['text'])} caractères\n")

entities = entity_extractor.extract_entities(test_doc['text'])
print(f"Nombre d'entités extraites: {len(entities)}")

# Afficher les entités par type
entity_types = {}
for entity in entities:
    entity_types[entity['label']] = entity_types.get(entity['label'], 0) + 1

print("\nRépartition des entités par type:")
for etype, count in sorted(entity_types.items(), key=lambda x: x[1], reverse=True):
    print(f"  {etype}: {count}")

# Afficher quelques exemples
print("\nExemples d'entités:")
for i, entity in enumerate(entities[:10], 1):
    print(f"  {i}. {entity['text']} ({entity['label']})")

## 3. Test d'extraction de relations

In [ ]:
# Initialiser l'extracteur de relations (sans LLM pour les tests)
relation_extractor = RelationExtractor(use_llm=False)

# Extraire les relations
relations = relation_extractor.extract_relations(test_doc['text'])
print(f"Nombre de relations extraites: {len(relations)}")

# Afficher quelques relations
print("\nExemples de relations:")
for i, rel in enumerate(relations[:10], 1):
    print(f"  {i}. {rel.get('subject', '?')} --[{rel.get('predicate', '?')}]--> {rel.get('object', '?')}")

## 4. Analyse sur l'ensemble des documents

In [ ]:
# Extraire les entités de tous les documents
print("Extraction des entités sur tous les documents...")
all_entities = entity_extractor.extract_from_documents(documents)

# Statistiques globales
total_entities = 0
entity_type_dist = {}

for doc_entities in all_entities:
    for entity in doc_entities['entities']:
        total_entities += 1
        entity_type_dist[entity['label']] = entity_type_dist.get(entity['label'], 0) + 1

print(f"\nStatistiques globales:")
print(f"Nombre total d'entités: {total_entities}")
print(f"Nombre moyen d'entités par document: {total_entities/len(documents):.1f}")

print("\nDistribution des types d'entités:")
for etype, count in sorted(entity_type_dist.items(), key=lambda x: x[1], reverse=True):
    percentage = (count / total_entities) * 100
    print(f"  {etype}: {count} ({percentage:.1f}%)")

## 5. Visualisation des résultats

In [ ]:
# Visualisation
fig, axes = plt.subplots(1, 2, figsize=(14, 6))

# Distribution des types d'entités
etype_df = pd.DataFrame(list(entity_type_dist.items()), columns=['Type', 'Count'])
etype_df = etype_df.sort_values('Count', ascending=False)
etype_df.plot(kind='bar', x='Type', y='Count', ax=axes[0], title='Distribution des types d\'entités')
axes[0].tick_params(axis='x', rotation=45)

# Entités les plus fréquentes
entity_counts = {}
for doc_entities in all_entities:
    for entity in doc_entities['entities']:
        key = (entity['text'].lower(), entity['label'])
        entity_counts[key] = entity_counts.get(key, 0) + 1

# Top 10 des entités les plus fréquentes
top_entities = sorted(entity_counts.items(), key=lambda x: x[1], reverse=True)[:10]
top_df = pd.DataFrame([(f"{text} ({label})", count) for (text, label), count in top_entities], 
                      columns=['Entity', 'Count'])
top_df.plot(kind='barh', x='Entity', y='Count', ax=axes[1], title='Top 10 des entités les plus fréquentes')

plt.tight_layout()
plt.show()

## 6. Sauvegarde des résultats

In [ ]:
# Sauvegarder les entités extraites
Path("../data/entities").mkdir(parents=True, exist_ok=True)
with open("../data/entities/entities.json", 'w') as f:
    json.dump(all_entities, f, ensure_ascii=False, indent=2)

print(f"Entités sauvegardées dans ../data/entities/entities.json")
print(f"Total: {len(all_entities)} documents avec entités")